In [6]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [7]:
files = [
    "MJD_Train_1.hdf5",
    "MJD_Train_2.hdf5",
    "MJD_Train_3.hdf5",
    "MJD_Train_4.hdf5"
]

In [8]:
def get_tdrift50(waveform, start_idx = 1000):
    # Find the index of the peak value
    max_idx = np.argmax(waveform)

    # Calculate the middle y-value (50%) between start and max
    start_y = waveform[start_idx]
    max_y = waveform[max_idx]
    mid_y = (start_y + max_y) / 2

    # Find the x-value (index) where the waveform crosses the middle y-value
    mid_x_idx = start_idx + np.argmax(waveform[start_idx:max_idx] >= mid_y)

    tdrift50 = mid_x_idx - start_idx 

    ## print(f" Start X value: {start_idx}\n",f"50% X value: {mid_x_idx}\n",f"tdrift50: {tdrift50}\n")

    return int(tdrift50)

In [9]:
def find_dcr(waveform):

    # Find peak index value
    peak_idx = np.argmax(waveform)

    # Find peak value
    peak_val = int(waveform[peak_idx])

    # We are only looking at the data after the peak
    data_after_peak = waveform[peak_idx:]

    # Get all time indices between peak and end of the time series
    time_indices = np.arange(peak_idx, len(waveform))

    # Calculate DCR region
    area_above_tail_slope = np.trapezoid(peak_val - data_after_peak, x=time_indices) 
    
    return area_above_tail_slope

In [10]:
for file_path in files:
    with h5py.File("data/" + file_path, 'r') as f:
        tdrift50_sum = 0
        dcr_sum = 0
        tdrift50_squared_sum = 0
        dcr_squared_sum = 0
        num_waveforms = 0

        raw_waveforms = np.array(f['raw_waveform'])

        for waveform in raw_waveforms:

            max_idx = np.argmax(waveform)
            
            # Skip waveform is it does not satisfy our conditions
            if waveform.size == 0:
                continue
            if 1000 >= len(waveform) or 1000 >= max_idx:
                continue

            # Calculate tdrift50 and DCR for the current waveform
            tdrift50 = get_tdrift50(waveform)
            dcr = find_dcr(waveform)

            # Accumulate sums for averages
            tdrift50_sum += tdrift50
            dcr_sum += dcr

            # Accumulate squared sums for MSE calculation
            tdrift50_squared_sum += tdrift50 ** 2
            dcr_squared_sum += dcr ** 2

            # Count waveforms
            num_waveforms += 1

        tdrift50_avg = tdrift50_sum / num_waveforms
        dcr_avg = dcr_sum / num_waveforms

        tdrift50_mse = (tdrift50_squared_sum / num_waveforms) - (tdrift50_avg ** 2)
        dcr_mse = (dcr_squared_sum / num_waveforms) - (dcr_avg ** 2)

        # Output the results
        print("Current File:", file_path)
        print("tdrift50 Average:", tdrift50_avg)
        print("tdrift50 MSE:", tdrift50_mse)
        print("DCR Average:", dcr_avg)
        print("DCR MSE:", dcr_mse)
        print("Number of Valid Waveforms in File:", num_waveforms)
        print("\n")


Current File: MJD_Train_1.hdf5
tdrift50 Average: 26.817876716546586
tdrift50 MSE: 1144.2832606927268
DCR Average: 744773.0910914696
DCR MSE: 593681133524.5258
Number of Valid Waveforms in File: 64956


Current File: MJD_Train_2.hdf5
tdrift50 Average: 26.82231595800363
tdrift50 MSE: 732.7296279255409
DCR Average: 751692.5256940061
DCR MSE: 602737236327.7921
Number of Valid Waveforms in File: 64958


Current File: MJD_Train_3.hdf5
tdrift50 Average: 26.61278357496845
tdrift50 MSE: 488.25177797215235
DCR Average: 753726.0061245653
DCR MSE: 606870795559.9003
Number of Valid Waveforms in File: 64974


Current File: MJD_Train_4.hdf5
tdrift50 Average: 26.879718578730216
tdrift50 MSE: 921.2698325213319
DCR Average: 748720.5308165335
DCR MSE: 596863613356.5417
Number of Valid Waveforms in File: 64956


